In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Analysis of stat

In [ ]:
stat_df = pd.read_csv('../result/experiment_rq_mle/stat_mle_RQ.csv')
stat_df

In [ ]:
stat_to_csv_df = {
    'model': [],
    'quantile': [],
    'loss': [],
    'mle\'s loss': [],
    'RQ\'s loss': [],
    'hit_rate_in': [],
    'mle\'s hit_rate_in': [],
    'RQ\'s hit_rate_in': [],
    'hit_rate_out': [],
    'mle\'s hit_rate_out': [],
    'RQ\'s hit_rate_out': [],
}

keys = [key for key in stat_to_csv_df][2:]

for q in stat_df['quantile'].unique():
    for mod in stat_df['model'].unique():
        cond1 = stat_df['quantile']==q
        cond2 = stat_df['model']==mod
        
        stat_to_csv_df['model'].append(mod)
        stat_to_csv_df['quantile'].append(q)
        
        for i, metric in enumerate(stat_df.columns[4:7]):
            try:
                x = stat_df[(stat_df['method']=='mle')&cond1&cond2][metric]
                y = stat_df[(stat_df['method']=='RQ')&cond1&cond2][metric]
    #             x = stat_df[(stat_df['method']=='mle')&(stat_df['best_start']==False)&cond1&cond2][metric]
    #             y = stat_df[(stat_df['method']=='RQ')&(stat_df['best_start']==False)&cond1&cond2][metric]
                if metric=='loss':
                    print('loss:', x.mean() < y.mean())
                if metric=='hit_rate_in':
                    print('hit_rate_in:', abs(0.05 - x.mean()) < abs(0.05 - y.mean()))
                if metric=='hit_rate_out':
                    print('hit_rate_out:', abs(0.05 - x.mean()) < abs(0.05 - y.mean()))
                stat_to_csv_df[keys[i*3]].append('accepted' if mannwhitneyu(x, y)[1] > 0.05 else 'rejected')
                stat_to_csv_df[keys[i*3+1]].append(np.round(x.mean(), 6))
                stat_to_csv_df[keys[i*3+2]].append(np.round(y.mean(), 6))
            except:
                # they are exactly the same 
                stat_to_csv_df[keys[i*3]].append('accepted')
                stat_to_csv_df[keys[i*3+1]].append(np.round(x.mean(), 6))
                stat_to_csv_df[keys[i*3+2]].append(np.round(y.mean(), 6))

In [ ]:
# pd.DataFrame(stat_to_csv_df).T.to_csv('../result/stat_test/stat_mle_RQ_analysis_res.csv')
pd.DataFrame(stat_to_csv_df).T

# Analysis of beta

In [ ]:
beta_df = pd.read_csv('../result/experiment_rq_mle/beta_mle_RQ.csv')
beta_df

In [ ]:
for model in beta_df.model.unique():
    for q in beta_df['quantile'].unique():
        plt.figure()
        sns.boxplot(
            data=beta_df[(beta_df['model']==model)&(beta_df['quantile']==q)],
            x='beta',
            y='coefficient',
            hue='method'
        )
        cond1 = beta_df['quantile']==q
        cond2 = beta_df['model']==mod
        plt.title(f'model: {model} and quantile: {q}')
        for beta in beta_df[beta_df['model']==model].beta.unique():
            cond3 = beta_df['beta']==beta
            x = beta_df[(beta_df['method']=='mle')&cond1&cond2&cond3]['coefficient']
            y = beta_df[(beta_df['method']=='RQ')&cond1&cond2&cond3]['coefficient']
            print(beta, mannwhitneyu(x, y)[1])        
        plt.show()

In [ ]:
beta_df.groupby(
    ['model','quantile','beta','method']
).mean()